In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded successfully")

Libraries loaded successfully


In [3]:
NIFTY_50_STOCKS = ['ADANIENT.NS', 'APOLLOHOSP.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BHARTIARTL.NS', 'BRITANNIA.NS', 'CIPLA.NS', 'COALINDIA.NS', 'DIVISLAB.NS', 'DRREDDY.NS', 'EICHERMOT.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS', 'HINDUNILVR.NS', 'ICICIBANK.NS', 'INDUSINDBK.NS', 'ITC.NS', 'JSWSTEEL.NS', 'KOTAKBANK.NS', 'LT.NS', 'M&M.NS', 'MARUTI.NS', 'NESTLEIND.NS', 'NTPC.NS', 'ONGC.NS', 'POWERGRID.NS', 'RELIANCE.NS', 'SBILIFE.NS', 'SBIN.NS', 'SUNPHARMA.NS', 'TCS.NS', 'TATACONSUM.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS', 'TECHM.NS', 'TITAN.NS', 'TRENT.NS', 'ULTRACEMCO.NS', 'UPL.NS', 'WIPRO.NS', 'ZEEL.NS', 'DMART.NS', 'ADANIPORTS.NS'
]
print(f"Total Nifty 50 stocks to analyze: {len(NIFTY_50_STOCKS)}")

Total Nifty 50 stocks to analyze: 50


In [14]:
# Analysing 6 Months Data to get top 20 liquid stocks from NIFTY_50_STOCKS
end_date = datetime.now()
start_date = end_date - timedelta(days=180) 
print(f"Analyzing data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print("="*80)
print("\nDownloading and analyzing liquidity...")

liquidity_data = []
failed_stocks = []

for idx, stock in enumerate(NIFTY_50_STOCKS, 1):
    try:
        stock_name = stock.replace('.NS', '')
        print(f"[{idx:2d}/{len(NIFTY_50_STOCKS)}] {stock_name:15s}", end=" ")
        
        # Download data
        data = yf.download(
            stock,
            start=start_date,
            end=end_date,
            progress=False
        )
        
        if len(data) < 50:  # Need at least 50 trading days
            print("✗ Insufficient data")
            failed_stocks.append(stock_name)
            continue
        
        # Calculate liquidity metrics
        close_price = data['Close']
        volume = data['Volume']
        
        # Daily turnover = Close Price × Volume (in crores)
        daily_turnover = (close_price * volume) / 10_000_000  # Convert to crores
        
        # Calculate metrics
        avg_turnover = daily_turnover.mean()
        median_turnover = daily_turnover.median()
        min_turnover = daily_turnover.min()
        max_turnover = daily_turnover.max()
        
        # Average volume
        avg_volume = volume.mean()
        
        # Latest price
        latest_price = close_price.iloc[-1]
        
        # Price range (for position sizing reference)
        price_range = close_price.max() - close_price.min()
        
        # Bid-ask spread proxy (High-Low as % of Close)
        spread_proxy = ((data['High'] - data['Low']) / close_price * 100).mean()
        
        liquidity_data.append({
        'Stock': stock_name,
        'Avg_Turnover_Cr': float(avg_turnover),   
        'Median_Turnover_Cr': float(median_turnover),
        'Min_Turnover_Cr': float(min_turnover),
        'Max_Turnover_Cr': float(max_turnover),
        'Avg_Volume': float(avg_volume),
        'Latest_Price': float(latest_price),
        'Avg_Spread_%': float(spread_proxy),
        'Trading_Days': int(len(data))
        })
        
        print(f"Avg: ₹{avg_turnover.item():6,.0f}Cr | Med: ₹{median_turnover.item():6,.0f}Cr")
        
    except Exception as e:
        print(f"Error: {str(e)[:40]}")
        failed_stocks.append(stock_name)

print("\n" + "="*80)
print(f"Successfully analyzed: {len(liquidity_data)} stocks")
if failed_stocks:
    print(f"✗ Failed: {failed_stocks}")

Analyzing data from 2025-04-15 to 2025-10-12

[ 1/50] ADANIENT        ✓ Avg: ₹   260Cr | Med: ₹   180Cr
[ 2/50] APOLLOHOSP      ✓ Avg: ₹   300Cr | Med: ₹   256Cr
[ 3/50] ASIANPAINT      ✓ Avg: ₹   263Cr | Med: ₹   217Cr
[ 4/50] AXISBANK        ✓ Avg: ₹   814Cr | Med: ₹   672Cr
[ 5/50] BAJAJ-AUTO      ✓ Avg: ₹   329Cr | Med: ₹   296Cr
[ 6/50] BAJFINANCE      ✓ Avg: ₹   797Cr | Med: ₹   662Cr
[ 7/50] BAJAJFINSV      ✓ Avg: ₹   288Cr | Med: ₹   228Cr
[ 8/50] BHARTIARTL      ✓ Avg: ₹ 1,286Cr | Med: ₹   970Cr
[ 9/50] BRITANNIA       ✓ Avg: ₹   173Cr | Med: ₹   147Cr
[10/50] CIPLA           ✓ Avg: ₹   219Cr | Med: ₹   183Cr
[11/50] COALINDIA       ✓ Avg: ₹   224Cr | Med: ₹   194Cr
[12/50] DIVISLAB        ✓ Avg: ₹   293Cr | Med: ₹   243Cr
[13/50] DRREDDY         ✓ Avg: ₹   206Cr | Med: ₹   178Cr
[14/50] EICHERMOT       ✓ Avg: ₹   291Cr | Med: ₹   252Cr
[15/50] GRASIM          ✓ Avg: ₹   180Cr | Med: ₹   152Cr
[16/50] HCLTECH         ✓ Avg: ₹   448Cr | Med: ₹   393Cr
[17/50] HDFCAMC         ✓ 